In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Train_df=pd.read_csv('/kaggle/input/gemstone/train.csv')
#Test_df=pd.read_csv('/kaggle/input/gemstone/test.csv')
Train_df=pd.read_csv(r'C:\Users\satish.hiremath\OneDrive - o9 Solutions\Desktop\python\e2e_mlops_dimond_price_prediction\e2e_mlops_dimond_price_prediction\artifacts\data_ingestion\train.csv')
Test_df=pd.read_csv(r'C:\Users\satish.hiremath\OneDrive - o9 Solutions\Desktop\python\e2e_mlops_dimond_price_prediction\e2e_mlops_dimond_price_prediction\artifacts\data_ingestion\test.csv')


In [ ]:
Train_df

In [ ]:
Test_df

In [ ]:
Train_df.columns

In [ ]:
Test_df.columns

In [ ]:
Train_df.isna().sum(),Test_df.isna().sum()

In [ ]:
Train_df.nunique()

In [ ]:
col=['cut','color','clarity']
def get_unique_value(columns,df):
    for i in columns:
        print(df[i].unique())

In [ ]:
def feature_engineering(df):
    df["volume"] = df["x"] * df["y"] * df["z"]
    df["surface_area"] = 2 * (df["x"] * df["y"] + df["y"] * df["z"] + df["z"] * df["x"])
    df["aspect_ratio_xy"] = df["x"] / df["y"]
    df["aspect_ratio_yz"] = df["y"] / df["z"]
    df["aspect_ratio_zx"] = df["z"] / df["x"]
    df["diagonal_distance"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2)
    df["relative_height"] = (df["z"] - df["z"].min()) / (df["z"].max() - df["z"].min())
    df["relative_position"] = (df["x"] + df["y"] + df["z"]) / (df["x"] + df["y"] + df["z"]).sum()
    df["volume_ratio"] = df["x"] * df["y"] * df["z"] / (df["x"].mean() * df["y"].mean() * df["z"].mean())
    df["length_ratio"] = df["x"] / df["x"].mean()
    df["width_ratio"] = df["y"] / df["y"].mean()
    df["height_ratio"] = df["z"] / df["z"].mean()
    df["sphericity"] = 1.4641 * (6 * df["volume"])**(2/3) / df["surface_area"]
    df["compactness"] = df["volume"]**(1/3) / df["x"]
    return df


In [ ]:
#df=feature_engineering(Train_df)


In [ ]:
df=Train_df

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [ ]:
Train_df

In [ ]:
df.drop(columns='id',inplace=True)

In [ ]:
df

In [ ]:
X = df.drop(columns=['price'],axis=1)


In [ ]:
X

In [ ]:
y=df['price']

In [ ]:
y

In [ ]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns



In [ ]:
num_features

In [ ]:
cat_features

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [ ]:
num_pipeline= Pipeline(
                steps=[
                ("imputer",SimpleImputer(strategy="median")),
                ("scaler",StandardScaler())

                ]
            )

cat_pipeline=Pipeline(

                steps=[
                ("imputer",SimpleImputer(strategy="most_frequent")),
                ("one_hot_encoder",OneHotEncoder()),
                ("scaler",StandardScaler(with_mean=False))
                ]

            )



preprocessor=ColumnTransformer(
                [
                ("num_pipeline",num_pipeline,num_features),
                ("cat_pipelines",cat_pipeline,cat_features)

                ]
        )

In [ ]:
X

In [ ]:
x_train=preprocessor.fit_transform(X)

In [ ]:
x_train[0]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

In [ ]:
label_encoding = LabelEncoder()
numeric_transformer = StandardScaler()

In [ ]:
def transform_data(data):
    for col in cat_features:
        data[col] = label_encoding.fit_transform(data[col].astype('str'))
    for col in num_features:
        data[col] = label_encoding.fit_transform(data[col].astype('float'))


In [ ]:
transform_data(X)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'target_column' is the name of your target column in your DataFrame
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [ ]:
pd.DataFrame(list(zip(y, y)), columns=['Name', 'Encoded'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [ ]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

In [ ]:
models = [  
    
    LinearRegression(),
    Lasso(),
    Ridge(),
    KNeighborsRegressor(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    XGBRegressor(), 
    CatBoostRegressor(verbose=False),
    AdaBoostRegressor(),
    
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]

In [ ]:
y


In [ ]:
models = {
            "Linear Regression": LinearRegression(),
            "K-Neighbors Regressor": KNeighborsRegressor(),
            "Decision Tree": DecisionTreeRegressor(),
            "Random Forest Regressor": RandomForestRegressor(),
            "XGBRegressor": XGBRegressor(), 
            "CatBoosting Regressor": CatBoostRegressor(verbose=False),
            "AdaBoost Regressor": AdaBoostRegressor()
            }
params={
                "Decision Tree": {
                    #'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                    # 'splitter':['best','random'],
                   #  'max_features':['sqrt','log2'],
                },
                "K-Neighbors Regressor": {},

                "Random Forest Regressor":{
                    #'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                 
                    # 'max_features':['sqrt','log2',None],
                    #'n_estimators': [8,16,32,64,128,256]
                },
                "Gradient Boosting":{
                   # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
                   # 'learning_rate':[.1,.01,.05,.001],
                    #'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
                    # 'criterion':['squared_error', 'friedman_mse'],
                    # 'max_features':['auto','sqrt','log2'],
                    #'n_estimators': [8,16,32,64,128,256]
                },
                "Linear Regression":{},
                "XGBRegressor":{
                 #   'learning_rate':[.1,.01,.05,.001],
                  #  'n_estimators': [8,16,32,64,128,256]
                },
                "CatBoosting Regressor":{
                  #  'depth': [6,8,10],
                  #  'learning_rate': [0.01, 0.05, 0.1],
                   # 'iterations': [30, 50, 100]
                },
                "AdaBoost Regressor":{
                    #'learning_rate':[.1,.01,0.5,.001],
                  #  'loss':['linear','square','exponential'],
                   # 'n_estimators': [8,16,32,64,128,256]
                }
                
            }


model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    para=params[list(models.keys())[i]]
    
    print(f"<<<<<<< Current Search Para {para} for the Model {model} <<<<<<<<")
    
    gs=GridSearchCV(model,para,cv=3)
    gs.fit(X_train,y_train)
    model.set_params(**gs.best_params_)
    
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.values())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    r2_list.append(model_train_r2)

    print('----------------------------------')
    
    print('Model performance for Test set')
    
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

In [ ]:
models
    

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)



In [ ]:
x_test=preprocessor.fit_transform(x_test)

In [ ]:
x_test

In [ ]:
Decission_model = RandomForestRegressor(criterion='poisson', max_features='log2',
                      n_estimators=500)
Decission_model = Decission_model.fit(x_train, y_train)
y_pred = Decission_model.predict(x_test)


In [ ]:
y_pred

In [ ]:
y=np.where(y_pred < 0.38, "False", "True")

In [ ]:
Train_df

In [ ]:
df = pd.DataFrame(zip(Test_df['PassengerId'], y), columns=['PassengerId', 'Transported'])


In [ ]:
df

In [ ]:
df.to_csv('MySubmissionV10.csv',index=False)